# Real Time Parametrized Aeroelastic Problem
## Author: Oriol CHANDRE VILA
## Tutors: Sylvain DUBREUIL (ONERA) et Joseph MORLIER (ISAE-SUPAERO)
### Internship in ONERA, within the Department of Systems and Information Treatment (DTIS).

The title of the internship is _Separated variables representation for the exploration of the Parametrized solution_. The methodology applied is based in an Offline-Online process, using an approach Greedy+POD for the phase offline and a Kriging interpolation in order to solve really fast cases in the Online phase (or real time phase).

This methodology has been applied to ONERA's code __aero_struct__ which computes the aeroelastic problem for a aircraft wing, type Airbus.

## I- OFFLINE PHASE
This program aims to use the POD-ROM to create a reduced basis able to be used in a real-time application.
It is applied to an aeroelastic problem. A Greedy algorithm has been used to introduced a POD based on 
the snapshots method combined with the Singular Value Decomposition (SVD).
Using version 3.0.6 of GMSH.

In [1]:
!pip install --upgrade pyDOE

Requirement already up-to-date: pyDOE in /Users/oriolchandrevila/anaconda2/lib/python2.7/site-packages (0.3.8)
Requirement not upgraded as not directly required: scipy in /Users/oriolchandrevila/anaconda2/lib/python2.7/site-packages (from pyDOE) (1.1.0)
Requirement not upgraded as not directly required: numpy in /Users/oriolchandrevila/anaconda2/lib/python2.7/site-packages (from pyDOE) (1.14.3)
distributed 1.21.8 requires msgpack, which is not installed.
grin 1.2.1 requires argparse>=1.1, which is not installed.
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import numpy as np
from scipy import linalg
import timeit
from sklearn.externals import joblib
from pyDOE import lhs
import functions_Offline as foff
from gaussian_process import GaussianProcess

### PARAMETERS BOUNDS: h_skins, h_ribs, h_spars_le, h_spars_te, b, S
h_* = thickness of different parts (m)

b = wing span (m)

S = wing surface (m^2)

- This parameters are the ones chosen to vary in our problem.

In [3]:
pMin_i = np.array([0.028,0.009,0.017,0.009,64.75,443.56])
pMax_i = np.array([0.031,0.015,0.025,0.015,75.65,500.21])
pMin = np.zeros((6,1))
pMin[:,0]= pMin_i[:]
pMax = np.zeros((6,1))
pMax[:,0]= pMax_i[:]

### OTHERS PARAMETERS OF INTEREST
alpha = angle of attack (°)

M_inf = Mach number

E = Young modulus (Pa)

nu = Poisson's ratio

BF = distance of the fuselage over the wing (m)

phi_d = sweep angle (°)

diedre_d = dihedral angle (°)

n_ribs_1 = number of ribs (first sector)

n_ribs_2 = number of ribs (second sector)

- This parameters are needed to define the geometry and material of the wing

In [4]:
alpha = 2.5 
M_inf = 0.85
E = 70.*1e9
nu = 0.3
BF = 5.96
phi_d = 30.
diedre_d = 1.5
n_ribs_1 =  8
n_ribs_2 =  12

## FIXED PARAMETERS (Flight Altitude = 11000 meters)
rho = density at 11000 meters (kg/m^3)

Pressure = pressure at 11000 meters (Pa)

adiabatic_i = adiabatic index of air

a_speed = speed of sound at 11000 meters (m/s)

phi = sweep angle (rad)

diedre = dihedral angle (rad)

In [5]:
rho = 0.3629
Pressure = 22552.
adiabiatic_i = 1.4
a_speed = np.sqrt(adiabiatic_i*Pressure/rho)
v_inf = M_inf*a_speed
phi = phi_d*np.pi/180.
diedre = diedre_d*np.pi/180.

Creation of the mesh files: VLM and FEM

In [6]:
def create_VLM_mesh(span_1,span_2,theta_1,theta_2,theta_3,L_1,c_1,c_2,c_3,phi_1,phi_2,d_1,d_2,n_1 = 12, n_2 = 20, n_3 = 39):
    #plan form is defined by 2 surfaces i.e 6 points
    points = np.zeros((6,3))
    l = c_1 - 0.25*L_1
    points[0,:] = [-L_1*0.25,0.0,np.tan(theta_1)*0.25*L_1]
    points[1,:] = [l,0.0,-np.tan(theta_1)*l]
    delta_z_1 = np.tan(d_1)*span_1
    points[2,:] = [np.tan(phi_1)*span_1 +0.75*c_2 ,span_1,delta_z_1-0.75*c_2*np.tan(theta_2)]
    points[3,:] = [np.tan(phi_1)*span_1-0.25*c_2 ,span_1,delta_z_1+0.25*c_2*np.tan(theta_2)]
    delta_z_2 = delta_z_1 + np.tan(d_2)*span_2
    delta_x = np.tan(phi_1)*span_1
    points[4,:] = [delta_x+np.tan(phi_2)*span_2+0.75*c_3,span_1+span_2,delta_z_2-0.75*c_3*np.tan(theta_3)]
    points[5,:] = [delta_x+np.tan(phi_2)*span_2-0.25*c_3,span_1+span_2,delta_z_2+0.25*c_2*np.tan(theta_3)]
    #Writing the .geo file for the vlm mesh
    f = open(r'param_wing/VLM_mesh.geo','w')
    for i in range(6):
            f.write('Point(100'+str(int(i+1))+') = {'+str(points[i,0])+','+str(points[i,1])+','+str(points[i,2])+',1e+5};\n')
    
    f.write('Line(100) = {1001,1002};\n')
    f.write('Line(200) = {1002,1003};\n')
    f.write('Line(300) = {1003,1004};\n')
    f.write('Line(400) = {1004,1001};\n')
    f.write('Line(500) = {1003,1005};\n')
    f.write('Line(600) = {1005,1006};\n')
    f.write('Line(700) = {1006,1004};\n')
    f.write('Physical Line("leading_edge") = {400,700};\n')
    f.write('Physical Line("trailing_edge") = {200,500};\n')
    f.write('Line Loop(100) = {100,200,300,400};\n')
    f.write('Line Loop(200) = {-300,500,600,700};\n')
    f.write('Ruled Surface(1) = {100};\n')
    f.write('Ruled Surface(2) = {200};\n')
    f.write('Physical Surface("wing") = {1, 2};\n')
    f.write('Transfinite Line {100,300,600} = '+str(n_1)+' Using Progression 1;\n')
    f.write('Transfinite Line {200,400} = '+str(n_2)+' Using Progression 1;\n')
    f.write('Transfinite Line {500,700} = '+str(n_3)+' Using Progression 1;\n')
    f.write('Transfinite Surface {1} = {1001, 1002, 1003, 1004};\n')
    f.write('Transfinite Surface {2} = {1004, 1003, 1005, 1006};\n')
    f.write('Recombine Surface {1,2};\n')
    f.close()

def create_FEM_mesh(VLM_geo,tck,chord_pos, n_ribs_1,n_ribs_2,n_x = 8,n_y = 2,n_z = 3):
    f = open(VLM_geo,'r')
    lines = f.readlines()
    f.close()
    points_geo = np.zeros((6,3))
    for i in range(6):
        line = lines[i].split()
        coord = line[2].split(',')
        points_geo[i,0] = float(coord[0].split('{')[1])
        points_geo[i,1] = float(coord[1])
        points_geo[i,2] = float(coord[2])
        
    #wing box is defined by 3 boxes located at the sections of the wing
    points = np.zeros((12,3))
    for i in range(3):
        points_A_B = points_geo[(2*i):(2*i)+2,:]
        ind = points_A_B[:,0].argsort()
        points_A_B = points_A_B[ind,:]
        vect = points_A_B[1]-points_A_B[0]
        A_s = points_A_B[0] + chord_pos[i,0]*vect
        B_s = points_A_B[0] + chord_pos[i,1]*vect
        points[(4*i):(4*i)+2,:] = A_s
        points[(4*i)+2:(4*i)+4,:] = B_s
        points[(4*i),2] = points[(4*i),2]+tck[i,0]/2.0 
        points[(4*i)+1,2] = points[(4*i)+1,2]-tck[i,0]/2.0
        points[(4*i)+2,2] = points[(4*i)+2,2]-tck[i,1]/2.0
        points[(4*i)+3,2] = points[(4*i)+3,2]+tck[i,1]/2.0
    
    #this boxes defined the front and rear spars and the upper and lower skin
    #definition of the ribs
    #vector 1-5
    v15 = points[4,:]-points[0,:]
    delta_15 = v15/n_ribs_1
    points_15 = np.zeros((n_ribs_1-1,3))
    for i in range(n_ribs_1-1):
        points_15[i,:] = points[0,:]+(i+1)*delta_15
    #vector 4-8
    v48 = points[7,:]-points[3,:]
    delta_48 = v48/n_ribs_1
    points_48 = np.zeros((n_ribs_1-1,3))
    for i in range(n_ribs_1-1):
        points_48[i,:] = points[3,:]+(i+1)*delta_48
    #vector 2-6
    v26 = points[5,:]-points[1,:]
    delta_26 = v26/n_ribs_1
    points_26 = np.zeros((n_ribs_1-1,3))
    for i in range(n_ribs_1-1):
        points_26[i,:] = points[1,:]+(i+1)*delta_26
    #vector 3-7
    v37 = points[6,:]-points[2,:]
    delta_37 = v37/n_ribs_1
    points_37 = np.zeros((n_ribs_1-1,3))
    for i in range(n_ribs_1-1):
        points_37[i,:] = points[2,:]+(i+1)*delta_37
        
    #vector 5-9
    v59 = points[8,:]-points[4,:]
    delta_59 = v59/n_ribs_2
    points_59 = np.zeros((n_ribs_2-1,3))
    for i in range(n_ribs_2-1):
        points_59[i,:] = points[4,:]+(i+1)*delta_59
    
    #vector 6-10
    v610 = points[9,:]-points[5,:]
    delta_610 = v610/n_ribs_2
    points_610 = np.zeros((n_ribs_2-1,3))
    for i in range(n_ribs_2-1):
        points_610[i,:] = points[5,:]+(i+1)*delta_610
    
    #vector 7-11
    v711 = points[10,:]-points[6,:]
    delta_711 = v711/n_ribs_2
    points_711 = np.zeros((n_ribs_2-1,3))
    for i in range(n_ribs_2-1):
        points_711[i,:] = points[6,:]+(i+1)*delta_711
    
    #vector 8-12
    v812 = points[11,:]-points[7,:]
    delta_812 = v812/n_ribs_2
    points_812 = np.zeros((n_ribs_2-1,3))
    for i in range(n_ribs_2-1):
        points_812[i,:] = points[7,:]+(i+1)*delta_812    
    
    #
    points_tot = np.zeros((12+4*(n_ribs_1-1)+4*(n_ribs_2-1),3))
    vects_1 = [points_15,points_48,points_26,points_37]
    points_tot[0:12,:] = points
    for i in range(4):
        points_tot[12+i*(n_ribs_1-1):12+(i+1)*(n_ribs_1-1),:] = vects_1[i] 
    vects_2 = [points_59,points_610,points_711,points_812]
    points_tot[0:12,:] = points
    for i in range(4):
        points_tot[12+4*(n_ribs_1-1)+i*(n_ribs_2-1):12+4*(n_ribs_1-1)+(i+1)*(n_ribs_2-1),:] = vects_2[i]
    
    ind = points_tot[:,1].argsort()
    points_tot = points_tot[ind]
    points_final = np.zeros(points_tot.shape)
    for i in range(points_tot.shape[0]/4):
        quad_points = points_tot[i*4:(i+1)*4,:]
        ind_z = quad_points[:,2].argsort()
        points_final[i*4,:] = quad_points[ind_z][2:,:][quad_points[ind_z][2:,0].argmin()]
        points_final[i*4+1,:] = quad_points[ind_z][:2,:][quad_points[ind_z][:2,0].argmin()]
        points_final[i*4+2,:] = quad_points[ind_z][:2,:][quad_points[ind_z][:2,0].argmax()]
        points_final[i*4+3,:] = quad_points[ind_z][2:,:][quad_points[ind_z][2:,0].argmax()]
    
        
    #number of mesh nodes
    #Writing the .geo file for the vlm mesh
    f = open(r'param_wing/FEM_mesh.geo','w')
    for i in range(points_final.shape[0]):
            f.write('Point('+str(int(i+1))+') = {'+str(points_final[i,0])+','+str(points_final[i,1])+','+str(points_final[i,2])+',1e+1};\n')
    #lines
    for i in range(points_final.shape[0]/4-1):
        f.write('Line('+str(8*i+1)+') = {'+str(4*i+1)+','+str(4*i+2)+'};\n')
        f.write('Line('+str(8*i+2)+') = {'+str(4*i+2)+','+str(4*i+3)+'};\n')
        f.write('Line('+str(8*i+3)+') = {'+str(4*i+3)+','+str(4*i+4)+'};\n')
        f.write('Line('+str(8*i+4)+') = {'+str(4*i+4)+','+str(4*i+1)+'};\n')
        
        f.write('Line('+str(8*i+5)+') = {'+str(4*i+1)+','+str(4*(i+1)+1)+'};\n')
        f.write('Line('+str(8*i+6)+') = {'+str(4*i+2)+','+str(4*(i+1)+2)+'};\n')
        f.write('Line('+str(8*i+7)+') = {'+str(4*i+3)+','+str(4*(i+1)+3)+'};\n')
        f.write('Line('+str(8*i+8)+') = {'+str(4*i+4)+','+str(4*(i+1)+4)+'};\n')
    
    #4 last lines
    ind = points_final.shape[0]/4-1
    f.write('Line('+str(8*ind+1)+') = {'+str(4*ind+1)+','+str(4*ind+2)+'};\n')
    f.write('Line('+str(8*ind+2)+') = {'+str(4*ind+2)+','+str(4*ind+3)+'};\n')
    f.write('Line('+str(8*ind+3)+') = {'+str(4*ind+3)+','+str(4*ind+4)+'};\n')
    f.write('Line('+str(8*ind+4)+') = {'+str(4*ind+4)+','+str(4*ind+1)+'};\n')
    
    #Surfaces
    skins = []
    spars_le = []
    spars_te = []
    ribs = []
    #5 surfaces per boxe
    for i in range(points_final.shape[0]/4-1):
        #skins
        f.write('Line Loop('+str(5*i+1)+') = {'+str(8*i+4)+','+str(8*i+5)+',-'+str(8*i+12)+',-'+str(8*i+8)+'};\n')
        f.write('Surface('+str(5*i+1)+') = {'+str(5*i+1)+'};\n')
        f.write('Line Loop('+str(5*i+2)+') = {'+str(8*i+2)+','+str(8*i+7)+',-'+str(8*i+10)+',-'+str(8*i+6)+'};\n')
        f.write('Surface('+str(5*i+2)+') = {'+str(5*i+2)+'};\n')  
        skins.extend([5*i+1,5*i+2])
        
        #spars
        f.write('Line Loop('+str(5*i+3)+') = {'+str(8*i+3)+','+str(8*i+8)+',-'+str(8*i+11)+',-'+str(8*i+7)+'};\n')
        f.write('Surface('+str(5*i+3)+') = {'+str(5*i+3)+'};\n')
        f.write('Line Loop('+str(5*i+4)+') = {'+str(8*i+1)+','+str(8*i+6)+',-'+str(8*i+9)+',-'+str(8*i+5)+'};\n')
        f.write('Surface('+str(5*i+4)+') = {'+str(5*i+4)+'};\n')    
        spars_te.extend([5*i+3])
        spars_le.extend([5*i+4])
        
        #ribs
        f.write('Line Loop('+str(5*i+5)+') = {'+str(8*i+10)+','+str(8*i+11)+','+str(8*i+12)+','+str(8*i+9)+'};\n')
        f.write('Surface('+str(5*i+5)+') = {'+str(5*i+5)+'};\n')    
        ribs.extend([5*i+5])
    
    f.write('Physical Surface("skins") = {')
    for i in range(len(skins)-1):
        f.write(str(skins[i])+',')
    f.write(str(skins[-1])+'};\n')
    
    f.write('Physical Surface("spars_le") = {')
    for i in range(len(spars_le)-1):
        f.write(str(spars_le[i])+',')
    f.write(str(spars_le[-1])+'};\n')  

    f.write('Physical Surface("spars_te") = {')
    for i in range(len(spars_te)-1):
        f.write(str(spars_te[i])+',')
    f.write(str(spars_te[-1])+'};\n')  
    
    f.write('Physical Surface("ribs") = {')
    for i in range(len(ribs)-1):
        f.write(str(ribs[i])+',')
    f.write(str(ribs[-1])+'};\n')  
    
    #mesh
    f.write('Transfinite Line {')
    for i in range(points_final.shape[0]/4-1):
        f.write(str(8*i+4)+','+str(8*i+2)+',')
    ind = points_final.shape[0]/4-1
    f.write(str(8*ind+4)+','+str(8*ind+2)+'} = '+str(n_x)+' Using Progression 1;\n')
    
    f.write('Transfinite Line {')
    for i in range(points_final.shape[0]/4-1):
        f.write(str(8*i+3)+','+str(8*i+1)+',')
    ind = points_final.shape[0]/4-1
    f.write(str(8*ind+3)+','+str(8*ind+1)+'} = '+str(n_z)+' Using Progression 1;\n')
    
    f.write('Transfinite Line {')
    for i in range(points_final.shape[0]/4-1):
        f.write(str(8*i+7)+','+str(8*i+8)+',')
    ind = points_final.shape[0]/4-1
    f.write(str(8*ind+7)+','+str(8*ind+8)+'} = '+str(n_y)+' Using Progression 1;\n')
    
    f.write('Transfinite Line {')
    for i in range(points_final.shape[0]/4-1):
        f.write(str(8*i+5)+','+str(8*i+6)+',')
    ind = points_final.shape[0]/4-1
    f.write(str(8*ind+5)+','+str(8*ind+6)+'} = '+str(n_y)+'Using Progression 1;\n')
    
    #structured mesh transfinite surface
    #5 surfaces per boxe
    for i in range(points_final.shape[0]/4-1):
        #skins
        f.write('Transfinite Surface('+str(5*i+1)+') = {'+str(4*i+1)+','+str(4*i+4)+',-'+str(4*i+8)+',-'+str(4*i+5)+'};\n')
        f.write('Transfinite Surface('+str(5*i+2)+') = {'+str(4*i+2)+','+str(4*i+3)+',-'+str(4*i+7)+',-'+str(4*i+6)+'};\n')
        #spars
        f.write('Transfinite Surface('+str(5*i+3)+') = {'+str(4*i+4)+','+str(4*i+3)+',-'+str(4*i+7)+',-'+str(4*i+8)+'};\n')
        f.write('Transfinite Surface('+str(5*i+4)+') = {'+str(4*i+1)+','+str(4*i+2)+',-'+str(4*i+6)+',-'+str(4*i+5)+'};\n')
        #rib
        f.write('Transfinite Surface('+str(5*i+5)+') = {'+str(4*i+5)+','+str(4*i+6)+',-'+str(4*i+7)+',-'+str(4*i+8)+'};\n')   
    
    f.close()

In [7]:
import subprocess

"""
Definition of a function which creates the .msh files.
"""
def create_mesh_files(n_ribs_1,n_ribs_2):
    VLM_geo = 'param_wing/VLM_mesh.geo'
    a = airbus_wing()
    span_1, span_2 =  a.compute_span()
    L_1, c_1, c_2, c_3 = a.compute_cord()
    e_1,e_2,e_3 = a.compute_ep() 
    d_1 = d_2 = a.compute_diedre()
    theta_1, theta_2, theta_3 = a.compute_twist()
    phi_1, phi_2  = a.compute_phi()
    coeff = 0.70
    tck = np.array([[e_1,coeff*e_1],[e_2,coeff*e_2],[e_3,coeff*e_3]])
    chord_pos = np.array([[0.10,0.8],[0.10,0.8],[0.10,0.8]])
    create_VLM_mesh(span_1,span_2,theta_1,theta_2,theta_3,L_1,c_1,c_2,c_3,phi_1,phi_2,d_1,d_2)
    create_FEM_mesh(VLM_geo,tck,chord_pos,n_ribs_1,n_ribs_2)
    FEM_geo = 'param_wing/FEM_mesh.geo'
    #Creating a file .msh
    subprocess.call(['gmsh', VLM_geo, '-2'])
    subprocess.call(['gmsh', FEM_geo, '-2'])

Initialisation of the problem

In [8]:
param_data = np.zeros(9)
param_data[0] = alpha
param_data[1] = v_inf
param_data[2] = rho
param_data[3] = E
param_data[4] = nu

### GREEDY ALGORITHM

In [9]:
from VLM import VLM_study
from FEM import FEM_study

"""
Definition of a function which returns the number of nodes
"""
def calcule_nodes(param_data):
    f = open('param_wing/FEM_mesh.msh','r')
    lines = f.readlines()
#    FEM_nodes = int(lines[lines.index('$Nodes\n')+1]) #Windows
    FEM_nodes = int(lines[lines.index('$Nodes\r\n')+1]) #Mac
    f.close()
    f = open('param_wing/VLM_mesh.msh','r')
    lines = f.readlines()
#    VLM_nodes = int(lines[lines.index('$Nodes\n')+1]) #Windows
    VLM_nodes = int(lines[lines.index('$Nodes\r\n')+1]) #Mac
    f.close()
    vlm_mesh_file = 'param_wing/new_VLM.msh'
    my_vlm = VLM_study(vlm_mesh_file,alpha = param_data[0],v_inf = param_data[1],rho = param_data[2])
    gamma_nodes = np.sum([my_vlm.surfaces[i]['points'].shape[0]for i in range(2)])
    return VLM_nodes, FEM_nodes, gamma_nodes

In the following step, the user defines the resolution of the problem: nSamples and nCandidates. In this example, they are fix at value 3 and 5 in order to get a fast result.

In [10]:
tic = timeit.default_timer()

# 0) Initialization
nSamples = 3
nIterations = nSamples-1
nCandidates = 5
nParam = len(pMin)
pCandidate = np.dot(pMin,np.ones((1,nCandidates))) + lhs(nParam,nCandidates).T*np.dot((pMax-pMin),np.ones((1,nCandidates)))
indexCand = np.zeros((nCandidates,), dtype=int)
for i in range(nCandidates):
    indexCand[i] = i
pCandMax = np.zeros((nParam,nSamples))

# 1) Ramdomly select a first sample (in the middle of the domain)
pMiddle = (pMin+pMax)/2
param_data[5] = pMiddle[0,0]
param_data[6] = pMiddle[1,0]
param_data[7] = pMiddle[2,0]
param_data[8] = pMiddle[3,0]
b = pMiddle[4,0]
S = pMiddle[5,0]
pCandMax[:,0] = pMiddle[:,0]
# Calculating the number of nodes and initialization of samples matrices
n_VLM_nodes, n_FEM_nodes, n_gamma_nodes = calcule_nodes(param_data)
uSamples = np.zeros((n_FEM_nodes*6,nSamples)) # POD displacement
gSamples = np.zeros((n_gamma_nodes,nSamples)) # POD circulation

# 2) Solving HDM-based problem
print "Greedy Iteration num.: 0\n",
uMiddle,gMiddle = foff.run_aerostruct(param_data,b,S,phi,diedre,BF,Mach=M_inf)
uSamples[:,0] = uMiddle
gSamples[:,0] = gMiddle

# 3) Building the ROB
uV,_,_ = linalg.svd(uSamples,full_matrices=False)
gV,_,_ = linalg.svd(gSamples,full_matrices=False)
uV_tr = np.transpose(uV)
gV_tr = np.transpose(gV)

# 4) Iteration loop
for iIteration in range(nIterations):
    print "Greedy Iteration num.: ",iIteration+1
    
    # 5) Solve argmax(residual)
    maxError = 0.
    candidateMaxError = 0
    for iCandidate in indexCand:
        # 5a) Loading candidates information
        param_data[5] = pCandidate[0,iCandidate]
        param_data[6] = pCandidate[1,iCandidate]
        param_data[7] = pCandidate[2,iCandidate]
        param_data[8] = pCandidate[3,iCandidate]
        b = pCandidate[4,iCandidate]
        S = pCandidate[5,iCandidate]
        A, B = foff.VLM_params(param_data,b,S,phi,diedre,BF,Mach=M_inf)
        Ar = np.dot(np.dot(gV_tr,A),gV)
        Br = np.dot(gV_tr,B)
        Ar_lu = linalg.lu_factor(Ar)
        gr = linalg.lu_solve(Ar_lu,-Br)
        g_exp = np.dot(gV,gr)
        F_s = foff.get_Fs(g_exp,param_data)
        K, Fs = foff.get_FEM(param_data,F_s)
        Kr = np.dot(np.dot(uV_tr,K),uV)
        Fr = np.dot(uV_tr,Fs)
        q = linalg.solve(Kr,Fr)
        # 5b) Compute error indicator
        Y1 = np.dot(K,np.dot(uV,q))- Fs
        errorIndicator = np.dot(np.transpose(Y1),Y1)
        if (iCandidate == 0 or maxError < errorIndicator):
            candidateMaxError = iCandidate
            maxError = errorIndicator
            
    # 6) Solving HDM-based problem for the candidate with the highest error indicator
    param_data[5] = pCandidate[0,candidateMaxError]
    param_data[6] = pCandidate[1,candidateMaxError]
    param_data[7] = pCandidate[2,candidateMaxError]
    param_data[8] = pCandidate[3,candidateMaxError]
    b = pCandidate[4,candidateMaxError]
    S = pCandidate[5,candidateMaxError]
    pCandMax[:,iIteration+1] = pCandidate[:,candidateMaxError]
    uIter,gIter = foff.run_aerostruct(param_data,b,S,phi,diedre,BF,Mach=M_inf)
    uSamples[:,iIteration+1] = uIter
    gSamples[:,iIteration+1] = gIter
    
    # 7) Building the ROB
    uV,_,_ = linalg.svd(uSamples[:,0:iIteration+2],full_matrices=False)
    gV,_,_ = linalg.svd(gSamples[:,0:iIteration+2],full_matrices=False)
    uV_tr = np.transpose(uV)
    gV_tr = np.transpose(gV)
    ##UPDATING indexCand: The current index must be removed!
    xC = 0
    while True:
        if indexCand[xC] == candidateMaxError:
            break
        xC += 1
    indexCand = np.delete(indexCand,xC)

toc = timeit.default_timer()
print("TOTAL COMPUTATION TIME: "+str((toc-tic)/60.)+" min")

Greedy Iteration num.: 0


rbf_poly.py:88: RuntimeWarning: divide by zero encountered in log
  result = r**2 * log(r)
rbf_poly.py:88: RuntimeWarning: invalid value encountered in multiply
  result = r**2 * log(r)


Greedy Iteration num.:  1
Greedy Iteration num.:  2
TOTAL COMPUTATION TIME: 7.30915496747 min


### KRIGING

In [12]:
# Initialization
tic = timeit.default_timer()
U_comp = uSamples
G_comp = gSamples

# Weight calculations
a_kri = np.zeros((nSamples,nSamples))
b_kri = np.zeros((nSamples,nSamples))
for i in range(nSamples):
    for j in range(nSamples):
        a_kri[j,i] = np.dot(U_comp[:,j].T,uV[:,i])
        b_kri[j,i] = np.dot(G_comp[:,j].T,gV[:,i])

# Trained solution
mean = "constant"
covariance = "squared_exponential"
theta_U = np.array([100000.0]*6)
theta_L = np.array([0.001]*6)
theta_0 = np.array([1.0]*6)
for i in range(nSamples):
    GP_u = GaussianProcess(regr = mean, corr = covariance,theta0 = theta_0,thetaL = theta_L, thetaU = theta_U)
    GP_u.fit(pCandMax.T, a_kri[:,i])
    print GP_u
    GP_g = GaussianProcess(regr = mean, corr = covariance,theta0 = theta_0,thetaL = theta_L, thetaU = theta_U)
    GP_g.fit(pCandMax.T, b_kri[:,i])
    print GP_g
    joblib.dump(GP_u, "results/Offline/GP_alpha_"+str(i)+".pkl")
    joblib.dump(GP_g, "results/Offline/GP_beta_"+str(i)+".pkl")
    
toc = timeit.default_timer()
print("KRIGING COMPUTATION TIME: "+str(toc-tic)+" s")

GaussianProcess(beta0=None,
        corr=<function squared_exponential at 0x1a15b50140>,
        normalize=True, nugget=array(2.22045e-15), optimizer='fmin_cobyla',
        random_start=1,
        random_state=<mtrand.RandomState object at 0x111379730>,
        regr=<function constant at 0x1a15b46d70>, storage_mode='full',
        theta0=array([[1., 1., 1., 1., 1., 1.]]),
        thetaL=array([[0.001, 0.001, 0.001, 0.001, 0.001, 0.001]]),
        thetaU=array([[100000., 100000., 100000., 100000., 100000., 100000.]]),
        verbose=False)
GaussianProcess(beta0=None,
        corr=<function squared_exponential at 0x1a15b50140>,
        normalize=True, nugget=array(2.22045e-15), optimizer='fmin_cobyla',
        random_start=1,
        random_state=<mtrand.RandomState object at 0x111379730>,
        regr=<function constant at 0x1a15b46d70>, storage_mode='full',
        theta0=array([[1., 1., 1., 1., 1., 1.]]),
        thetaL=array([[0.001, 0.001, 0.001, 0.001, 0.001, 0.001]]),
        the

## II- ONLINE PHASE
Using the data saved within the two Kriging structures, real-time solutions can be computed in a really fast way.

In [13]:
from mesh_deformation_airbus import mesh_deformation_airbus

From the Offline Phase we will use the parameters: _nSamples_, _param data_, _uV_ and _gV_. Besides, the data of the Kriging structures.

### Definition of the case
Here, the user choose the values of his analysis!

In [14]:
h_skins = 0.028
h_ribs = 0.009
h_spars_le = 0.017
h_spars_te = 0.009
b = 64.75
S = 443.56

Initialisation 

In [15]:
tic = timeit.default_timer()
x_conf = np.array([h_skins,h_ribs,h_spars_le,h_spars_te,b,S])
pC = np.zeros((6))
pC[0] = param_data[5] = h_skins
pC[1] = param_data[6] = h_ribs
pC[2] = param_data[7] = h_spars_le
pC[3] = param_data[8] = h_spars_te
pC[4] = b
pC[5] = S

1) Kriging prediction of the interested point

In [16]:
u_mp = np.zeros((1,nSamples))
u_vp = np.zeros((1,nSamples))
gamma_mp = np.zeros((1,nSamples))
gamma_vp = np.zeros((1,nSamples))
for i in range(nSamples):
    # *Loading Kriging data
    name_a = joblib.load("results/Offline/GP_alpha_"+str(i)+".pkl")
    name_b = joblib.load("results/Offline/GP_beta_"+str(i)+".pkl")
    # **Prediction
    u_mp[0,i], u_vp[0,i] = name_a.predict(x_conf.reshape((1,6)), eval_MSE=True)
    gamma_mp[0,i], gamma_vp[0,i] = name_b.predict(x_conf.reshape((1,6)), eval_MSE=True)
u_mean = 0
u_var2 = 0
gamma_mean = 0
gamma_var2 = 0
for i in range(nSamples):
    u_mean += np.dot(u_mp[0,i],uV[:,i])
    u_var2 += np.dot(u_vp[0,i]**2,uV[:,i]**2)
    gamma_mean += np.dot(gamma_mp[0,i],gV[:,i])
    gamma_var2 += np.dot(gamma_vp[0,i]**2,gV[:,i]**2)

2) Calculation of u_est and g_est

In [17]:
u_est = u_mean
u_est1 = u_mean+3.0*np.sqrt(u_var2)
u_est2 = u_mean-3.0*np.sqrt(u_var2)
g_est = gamma_mean
g_est1 = gamma_mean+3.0*np.sqrt(gamma_var2)
g_est2 = gamma_mean-3.0*np.sqrt(gamma_var2)
toc = timeit.default_timer()
print("ONLINE COMPUTATION TIME: "+str(toc-tic)+" s")

ONLINE COMPUTATION TIME: 0.127323150635 s


3) Publication of the results: u_est, g_est

In [18]:
a_new = mesh_deformation_airbus('param_wing/VLM_mesh.msh','param_wing/FEM_mesh.msh') 
a_new.new_wing_mesh(b,S)
vlm_mesh = 'param_wing/new_VLM.msh'
fem_mesh = 'param_wing/new_FEM.msh'
n_VLM_nodes, n_FEM_nodes, n_gamma_nodes = calcule_nodes(pC)

# 3a) Computation of strains and stress
element_property,material,element_type = foff.create_dict(param_data)
my_fem = FEM_study(fem_mesh,element_type,element_property,material)
my_fem.read_mesh_file()
strain_dict, stress_dict = my_fem.get_strain_and_stress(u_est)

# 3b) u_est
my_fem.post_processing(u_est,"results/Param_wing/u_est")
my_fem.post_processing_var1(u_est1,"results/Param_wing/u_est1")
my_fem.post_processing_var2(u_est2,"results/Param_wing/u_est2")

## 3c) Von Mises stress
Von_Mises_Stress = my_fem.get_Von_Mises(stress_dict)
my_fem.post_processing_Von_Mises(Von_Mises_Stress,'results/param_wing/result_VM_iter')
print("Average Von Mises Stress = "+str(np.mean(Von_Mises_Stress))+" Pa")
stress_max = 0.
stress_max_i = 0
for i in range(len(Von_Mises_Stress)):
    stress_i = Von_Mises_Stress[i]
    if (stress_max < stress_i):
        stress_max = stress_i
        stress_max_i = i
print("Node VM max = "+str(stress_max_i))
print("Von Mises Stress max = "+str(Von_Mises_Stress[stress_max_i])+" Pa")

# 3d) g_est
my_vlm = VLM_study(vlm_mesh)
my_vlm.post_processing_gamma('Param_wing/g_est',g_est)
my_vlm.post_processing_gamma_var1('Param_wing/g_est1',g_est1)
my_vlm.post_processing_gamma_var2('Param_wing/g_est2',g_est2)

Average Von Mises Stress = 10506550.236048812 Pa
Node VM max = 483
Von Mises Stress max = 50253012.89419359 Pa


4) Quality of the prediction

In [19]:
for x in range(len(u_mean)):
    if u_mean[x] == 0:
        u_mean[x] = 1.*1e-35
tau_u = abs(np.sqrt(u_var2)/u_mean)
tau_g = abs(np.sqrt(gamma_var2)/gamma_mean)
my_fem.post_processing_erreur(tau_u,"results/Param_wing/tau_u")
my_vlm.post_processing_erreur('Param_wing/tau_g',tau_g)

5) Opening GMSH files

In [2]:
import subprocess
subprocess.call(['gmsh','results/Param_wing/g_est.msh','results/Param_wing/g_est1.msh','results/Param_wing/g_est2.msh','results/Param_wing/tau_g.msh','-open'])
subprocess.call(['gmsh','results/Param_wing/u_est.msh','results/Param_wing/u_est1.msh','results/Param_wing/u_est2.msh','results/Param_wing/tau_u.msh','-open'])
subprocess.call(['gmsh','results/Param_wing/result_VM_iter.msh','-open'])

OSError: [Errno 2] No such file or directory